<a href="https://colab.research.google.com/github/Francrodi/AprendAut_T2/blob/master/Cliente_Lab2_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 2 - Aprendizaje Bayesiano

### Grupo 50:
    - M. Ramilo (5.665.788-5)
    - G. Dinello (5.031.022-5)
    - F. Crocamo (4.732.816-6)

## 1. Objetivo

El objetivo central de esta tarea es crear un **predictor de palabras** que funcione de manera similar a las sugerencias de palabras que encontramos en los teclados de dispositivos celulares, y para lograrlo, vamos a emplear el algoritmo **Naive Bayes**. Para entrenar este modelo, utilizaremos un chat extenso extraído de **WhatsApp**. Esta elección de datos presenta desafíos particulares de preprocesamiento debido a la naturaleza informal y variada de las conversaciones en WhatsApp.

En esencia, estamos buscando desarrollar una herramienta que sea capaz de **predecir la siguiente palabra** basándose en el contexto de lo que se ha escrito previamente. Esto puede ser especialmente útil en la escritura de mensajes rápidos y eficientes en dispositivos móviles, ya que ahorra tiempo al usuario al sugerir las palabras más probables de acuerdo con el contexto.

## 2. Preprocesamiento

Como se mencionó anteriormente, esta sección del proyecto se centra en la etapa de preprocesamiento de los datos, que es una fase fundamental para optimizar el chat original extraído de WhatsApp y prepararlo adecuadamente para su uso en el entrenamiento del modelo de predicción de palabras. Esta etapa desempeña un papel esencial en la calidad y eficacia del predictor de palabras que estamos desarrollando.

### 2.1 Cargamos el chat

In [ ]:
filename = "chat.txt"

with open(filename, "r", encoding="utf-8") as file:
  data = file.readlines()

raw_chat = []

for line in data:
  parts = line.split(":")
  message = parts[-1].strip()
  raw_chat.append(message)

### 2.2 Preprocesamos el chat

Preprocesamos el chat para eliminar todos los elementos que disminuyen la calidad del predictor que queremos construir.

Los elementos filtrados son:

*   URLs
*   Menciones ("@alguien")
*   Risas
*   Caracteres especiales
*   Palabras con letras repetidas ("vamosss", "biennnn", etc...)

Se aplicaron transformaciones a abreviaciones de palabras para mantener consistencia en predicciones. Algunos ejemplos son:

*   "tmb" a "también"
*   "xq" a "porque"
*   "q" a "que"


In [ ]:
import re

# Expresiones regulares
URL = r"https?://\S+"
MENTION = r"@\S+"
LAUGH = r"(?:[aA]*(?:[jJ][aA]*)+[jJ]?)\b"
NOT_LETTERS = r"[^a-zñáéíóú\s]"
REPEATED_LETTERS = r"([a-zñáéíóú])\1{2,}"

# Mapeo de palabras informales a palabras formales
INFORMAL_TO_FORMAL = {
  "q": "que",
  "ke": "que",
  "k": "que",
  "xq": "porque",
  "porq": "porque",
  "pq": "porque",
  "p": "para",
  "pa": "para",
  "d": "de",
  "n": "en",
  "tb": "también",
  "tmb": "también",
  "tmbn": "también",
  "tmbien": "también",
  "tmbn": "también",
  "tambn": "también",
  "toy": "estoy",
  "tas": "estas",
  "ta": "está",
  "tan": "estan",
  "tamos": "estamos",
  "tamo": "estamos",
  "pal": "para el",
  "pa la": "para la",
  "pa las": "para las",
  "pa los": "para los",
  "vamo": "vamos",
  "bue": "bueno",
  "boe": "bueno",
  "buee": "bueno",
  "bn": "bien",
}


def preprocess_chat(chat: list) -> list:
  """Preprocesa el chat de WhatsApp, convirtiendo todas las letras en minusculas, eliminando URLs, menciones, risas, caracteres especiales, letras repetidas y mensajes vacios. Reemplaza palabras informales por sus equivalentes formales. Finalmente, ordena los mensajes alfabeticamente y elimina duplicados.

  Args:
    chat (list): Lista de mensajes del chat de WhatsApp

  Returns:
    list: Lista de mensajes preprocesados
  """
  chat = [message.lower() for message in chat]

  chat = [re.sub(URL, "", message) for message in chat]
  chat = [re.sub(MENTION, "", message) for message in chat]
  chat = [re.sub(NOT_LETTERS, "", message) for message in chat]
  chat = [re.sub(LAUGH, "", message) for message in chat]
  chat = [re.sub(REPEATED_LETTERS, r"\1", message) for message in chat]

  for informal, formal in INFORMAL_TO_FORMAL.items():
    chat = [re.sub(r"\b{}\b".format(informal), formal, message) for message in chat]

  chat = [message.strip() for message in chat]
  chat = [message for message in chat if message != ""]

  chat = [re.sub(r"\s+", " ", message) for message in chat]
  chat = list(dict.fromkeys(chat))

  return chat

chat = preprocess_chat(raw_chat)
chat

['los mensajes y las llamadas están cifrados de extremo a extremo nadie fuera de este chat ni siquiera whatsapp puede leerlos ni escucharlos toca para obtener más información',
 'creó el grupo métodos numéricos fing',
 'cualquier miembro de esta comunidad puede unirse a este grupo',
 'te uniste a un grupo en la comunidad matemática fing mediante una invitación',
 'se unió desde la comunidad',
 'se unió usando el enlace de invitación de este grupo',
 'buenas a alguien le falta grupo con un amigo somos',
 'a mi',
 'a mi también',
 'grupo para que estoy re perdido',
 'el lab es en grupo',
 'se',
 'mañana ya hay clase',
 'es de a cuatro el lab no',
 'o',
 'exacto',
 'yo estoy en un grupo de',
 'si queres los juntamos',
 'estoy sin grupo por si alguien quiere armar',
 'yo estoyy',
 'ya hay que armar los grupos',
 'cuando quieras pero ya se sabe que las actividades son en grupo',
 'fa bueno yo tampoco tengo grupo asi que si alguien quiere armamos',
 'estoy',
 'el trabajo obligatorio es en gr

## 3. Estructuras de datos

Para lograr un funcionamiento eficiente debemos crear las estructuras de datos adecuadas para el problema a resolver. A la hora de correr el predictor, necesitaremos tiempos de busqueda casi instantaneos, de otra manera el predictor en lugar de ayudar al usuario lo estaria trancando y estorbando.

Para lograr esto, utilizaremos **tablas de hash (diccionarios en Python)** para almacenar las probabilidades y cantidad de ocurrencia de palabras, esto nos permite realizar las busquedas en **O(1)**.

Por otro lado, construiremos una lista para almacenar el vocabulario, ya que en cada prediccion a realizar tendremos que recorrer todo el vocabulario en busca de la palabra mas probable (Hipotesis MAP).

De esta forma podemos conseguir tiempos de prediccion de **O(N)** siendo N la cantidad de palabras en el vocabulario.

### 3.1 Cargamos diccionario de palabras en español

El mismo sera utilizado para cruzarlo con nuestro vocabulario generado a partir del chat y asi eliminar palabras no deseadas.

In [ ]:
filename = "es.txt"

with open(filename, "r", encoding="utf-8") as file:
  data = file.readlines()

spanish_dict = set()

for line in data:
  word = line.strip()
  spanish_dict.add(word)

len(spanish_dict)

636598

### 3.2 Creamos diccionario de ocurrencia de palabras y vocabulario

In [ ]:
words_list = " ".join(chat).split(" ")

# filtrado de palabras con diccionario
filtered_words_list = [word for word in words_list if word in spanish_dict]

# conteo de ocurrencias de cada palabra
words_count = {}
for word in filtered_words_list:
    if word in words_count:
        words_count[word] += 1
    else:
        words_count[word] = 1

# vocabulario
vocabulary = list(words_count.keys())

In [ ]:
words_count

{'los': 150,
 'mensajes': 4,
 'las': 178,
 'llamadas': 1,
 'cifrados': 1,
 'de': 679,
 'extremo': 2,
 'nadie': 9,
 'fuera': 4,
 'este': 36,
 'ni': 33,
 'siquiera': 6,
 'puede': 47,
 'toca': 2,
 'para': 196,
 'obtener': 6,
 'el': 673,
 'grupo': 102,
 'cualquier': 7,
 'miembro': 1,
 'esta': 76,
 'comunidad': 6,
 'te': 146,
 'uniste': 1,
 'un': 139,
 'en': 434,
 'la': 508,
 'mediante': 1,
 'una': 130,
 'se': 187,
 'desde': 9,
 'usando': 12,
 'enlace': 2,
 'buenas': 72,
 'alguien': 85,
 'le': 78,
 'falta': 25,
 'con': 168,
 'amigo': 13,
 'somos': 18,
 'mi': 59,
 'que': 822,
 'estoy': 55,
 're': 22,
 'perdido': 6,
 'es': 343,
 'mañana': 21,
 'ya': 69,
 'hay': 75,
 'clase': 40,
 'cuatro': 1,
 'no': 458,
 'exacto': 5,
 'yo': 161,
 'si': 309,
 'queres': 6,
 'juntamos': 1,
 'sin': 22,
 'por': 148,
 'quiere': 8,
 'armar': 8,
 'grupos': 13,
 'cuando': 40,
 'quieras': 3,
 'pero': 172,
 'sabe': 20,
 'actividades': 1,
 'son': 64,
 'fa': 5,
 'bueno': 26,
 'tampoco': 13,
 'tengo': 38,
 'asi': 20,
 'ar

### 3.3 Definimos funciones para manejar los diccionarios de probabilidades

Definimos las funciones necesarias para construir y actualizar los diccionarios que almacenan las probabilidades de **P(H)** y **P(D|H)**.

In [ ]:
total_words = sum(words_count.values())

freq_ph = lambda frequency: frequency / total_words
laplace_ph = lambda frequency: (frequency + 1) / (total_words + len(vocabulary))

# Construimos diccionario de probabilidades P(H), probabilidad a priori
def build_ph(method="frequency") -> dict[str, float]:
    if method == "frequency":
        ph = freq_ph
    if method == "laplace":
        ph = laplace_ph

    ph_dict = {word: ph(frequency) for word, frequency in words_count.items()}
    return ph_dict

# Construimos diccionario de probabilidades P(D|H) en base al hiperparametro N (Cantidad de palabras a considerar)
def build_pdh(chat: list, N=2) -> dict[str, dict[str, float]]:
    pdh_dict = {}
    for message in chat:
        words = message.split(" ")
        # FIX de no encuentra "y", ver si al final vamos a filtrar o no
        words = [word for word in words if word in vocabulary]
        if len(words) > N: # Si el mensaje tiene menos de N palabras no se toma en cuenta. TODO: ver si tiene sentido o usar > 1
            for key, val in enumerate(words):
                if (val not in pdh_dict.keys()):
                    pdh_dict[val] = {}
                if (key >= N):
                    # https://eva.fing.edu.uy/mod/forum/discuss.php?d=275096
                    # "sería cantidad de instancias en que aparece la palabra "olvidó" en las primeras N palabras (sin contar las apariciones, es binario: aparece o no aparece) "
                    # ocurrencia de H=word_before antes de D=val
                    N_words_before = set(words[key-N:key])
                    for word_before in N_words_before:
                       if not word_before in pdh_dict[val]:
                           pdh_dict[val][word_before] = 0
                       pdh_dict[val][word_before] += 1
                    # version anterior
                    # for word_before in words[key-N:key]:
                    #     if not word_before in pdh_dict[val]:
                    #         pdh_dict[val][word_before] = 0
                    #     pdh_dict[val][word_before] += 1

    # probabilidad de palabra subkey dado key
    for key in pdh_dict.keys():
        for subkey in pdh_dict[key].keys():

            if not (0 <= (pdh_dict[key][subkey]/words_count[subkey]) <= 1):
                print(f"Warning: pdh_dict[{key}][{subkey}] = {pdh_dict[key][subkey]} is not between 0 and 1.")
                print(f"words_count[{subkey}] = {words_count[subkey]}")

            # https://eva.fing.edu.uy/mod/forum/discuss.php?d=275096
            # habria que revisar que no hayan prob > 1?
            pdh_dict[key][subkey] /= words_count[subkey]

    return pdh_dict


def update_ph(word: str):
    """ Update the prior probability distribution P(H) based on the occurrence of a new word."""
    global total_words, ph_dict, words_count
    total_words += 1
    if word in words_count:
        words_count[word] += 1
    else:
        words_count[word] = 1
    if word in ph_dict:
        ph_dict[word] = words_count[word] / total_words
    else:
        ph_dict[word] = 1 / total_words

def update_pdh(message: str, pdh: dict, N=2, method="frequency"):
    # """ Update the conditional probability distribution P(D|H) based on the occurrence of a new message.
    # It only updates the last word in the message as this function will be called for each new message in the chat.""
    # """
    # words = message.split(" ")
    # target_word = words[-1]
    # if len(words) >= N:
    #     context_words = words[-N:-1]
    # else:
    #     context_words = words[:-1]

    # if method == "frequency":
    #     ph = freq_ph
    # if method == "laplace":
    #     ph = laplace_ph

    # for context_word in context_words:
    #     pdh[target_word][context_word] = (pdh[target_word][context_word] + 1) / (words_count[context_word] + 1)
    pass


## 4. Implementación

In [ ]:
import nltk
nltk.download('stopwords')


# Definimos stopwords
stopwords = nltk.corpus.stopwords.words("spanish")
stopwords.append("no")
stopwords.append("si")
stopwords.append("mas")
stopwords.append("que")

# Definimos hiperparametro N
N = 3

# Construimos los diccionarios de probabilidades
ph_dict = build_ph()
for prob in ph_dict.values():
       assert(0 <= prob <= 1)

pdh_dict = build_pdh(chat, N)
update_pdh = lambda message: update_pdh(message, pdh_dict, N)


def recomendacion_bayesiana(frase):
  if len(frase) >= N:
    keys = frase[-N:]
  else:
    keys = frase

  keys = [k.lower() for k in keys]

  # Bucar la palabra con mayor probabilidad, es decir, que maximice P(H|D) = P(D|H) * P(H) (Hipoteses MAP)
  max_prob = 0
  palabra_sugerida = ""
  last_is_stopword = True if keys[-1] in stopwords else False
  for word in vocabulary:
    if last_is_stopword and word in stopwords:
      continue
    prob = None
    for key in keys:
      m_estimator = 1 / (words_count.get(key, 0) + len(vocabulary))
      prob = pdh_dict.get(word, {}).get(key, m_estimator) if prob is None else prob * pdh_dict.get(word, {}).get(key, m_estimator)
    prob *= ph_dict[word]
    if prob >= max_prob:
      max_prob = prob
      palabra_sugerida = word

  return palabra_sugerida


##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")
frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      for palabra in frase:
        update_ph(palabra) # Se actualiza el diccionario para P(H)

      update_pdh(frase)
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyError: ignored

# ideas

- normalizar P(H) utilizando el suavizado de Laplace o suavizado de add-one (también conocido como suavizado de Laplace). Garantiza que ninguna probabilidad sea cero y que todas las palabras tengan una probabilidad no nula de ocurrencia.
P(H) = (frequency(H) + 1) / (total_words + V)
Donde:

    frequency(H) es la frecuencia de la palabra H en el corpus.
    total_words es el número total de palabras en el corpus.
    V es el tamaño del vocabulario (el número total de palabras únicas en el corpus).
El valor 1 se agrega a la frecuencia para evitar probabilidades de cero. Sin embargo, el valor exacto de la constante de suavizado (en este caso, 1) puede ajustarse según las necesidades específicas de tu modelo y el tamaño del corpus.
[Podriamos entrenar con y sin y evaluar si mejora o empeora]

In [ ]:
# Usando TF-IDF
# https://eva.fing.edu.uy/mod/forum/discuss.php?d=275273

from collections import defaultdict
import math

total_messages = len(chat)  # Total number of messages

# TF-IDF for a word in a message
def calculate_tfidf(word, message, chat):
    tf = message.count(word) / len(message.split())  # Term Frequency (TF)
    idf = math.log(total_messages / (sum(1 for msg in chat if word in msg) + 1))  # Inverse Document Frequency (IDF)
    return tf * idf

# Initialize dictionaries for P(H) and P(D|H)
ph_dict = {}  # P(H)
pdh_dict = defaultdict(dict)  # P(D|H), defaultdict(dict): subclase de dict, cuando no encuentra una key no devuelve error sino que agrega un {}

# Calculate P(H) using TF-IDF
def build_ph() -> dict:
    for word in words_dict:
        ph_dict[word] = sum(calculate_tfidf(word, message, chat) for message in chat) / total_words
    return ph_dict

# Calculate P(D|H) using TF-IDF
def build_pdh(chat: list, N=2) -> dict:
    for message in chat:
        words = message.split(" ")
        if len(words) > N:
            for key, val in enumerate(words):
                if val not in pdh_dict:
                    pdh_dict[val] = {}
                if key >= N:
                    N_words_before = words[key - N:key]
                    for word_before in N_words_before:
                        pdh_dict[val][word_before] = calculate_tfidf(val, message, chat)
    return pdh_dict

# Update P(H) and P(D|H) when a new word is encountered
def update_ph(word: str):
    global total_words, ph_dict
    total_words += 1
    ph_dict[word] = sum(calculate_tfidf(word, message, chat) for message in chat) / total_words

def update_pdh(message: str, pdh: dict):
    global pdh_dict
    words = message.split(" ")
    for key, val in enumerate(words):
        if val not in pdh_dict:
            pdh_dict[val] = {}
        if key >= N:
            N_words_before = words[key - N:key]
            for word_before in N_words_before:
                pdh_dict[val][word_before] = calculate_tfidf(val, message, chat)

# Initialize P(H) and P(D|H)
ph_dict = build_ph()
pdh_dict = build_pdh(chat, N)


- evaluacion tomando K frases random de N palabras del dataset y evaluando la prediccion basado en las N-1